In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

<h1>matchTemplate</h1>

In [2]:
src = cv.imread('/Users/kolsha/Pictures/moments/multiple-blob.png')
#src_gray = cv.medianBlur(src_gray, 5)
src = cv.cvtColor(src, cv.COLOR_BGR2RGB)


methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
noises = ['None','gauss', 's&p', 'poisson', 'speckle']

def noisy(noise_typ,image):
    if noise_typ == "gauss":
        row,col,ch= image.shape
        mean = 0
        var = 0.1
        sigma = var**0.5
        gauss = np.random.normal(mean,sigma,(row,col,ch))
        gauss = gauss.reshape(row,col,ch)
        noisy = image + gauss
        return noisy
    elif noise_typ == "s&p":
        row,col,ch = image.shape
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        # Salt mode
        num_salt = np.ceil(amount * image.size * s_vs_p)
        coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
        out[tuple(coords)] = 1

        # Pepper mode
        num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
        coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
        out[tuple(coords)] = 0
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ =="speckle":
        row,col,ch = image.shape
        gauss = np.random.randn(row,col,ch)
        gauss = gauss.reshape(row,col,ch)        
        noisy = image + image * gauss
        return noisy
    
    return image



@interact(method=methods, image=(1,6), noise=noises, rotate=(0, 360), scale=(0.5, 1.5, 1e-2))
def match_tpl(method, image, noise, rotate, scale):
    method = eval(method)
    tpl = cv.imread('/Users/kolsha/Pictures/moments/'+ str(image) + '.jpg')
    
    img = src.copy()
    
    img = noisy(noise, img)
    img = img.astype('uint8')
    #print(img.shape, img.max())
    src_gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
    
    num_rows, num_cols = tpl.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    tpl = cv.warpAffine(tpl, rotation_matrix, (num_cols, num_rows))
    
    tpl_gray = cv.cvtColor(tpl, cv.COLOR_BGR2GRAY)
    tpl = cv.cvtColor(tpl, cv.COLOR_BGR2RGB)
    
    res = cv.matchTemplate(src_gray,tpl_gray,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    
  
    dst_src = np.copy(img)
    #dst_tpl = np.copy(tpl)
    
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + num_cols, top_left[1] + num_rows)

    cv.rectangle(dst_src,top_left, bottom_right, 255, 2)

 
    
    titles = ['C Image', 'C Tpl']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [dst_src, tpl]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 15.5)
    for i in range(len(images)):
        plt.subplot(2, 3, i+1),
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (method, image, rotate, scale)
    
    
    
    

interactive(children=(Dropdown(description='method', options=('cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CC…